In [1]:
#Librerias
import folium
from folium.plugins import MarkerCluster, Search
import pandas as pd
import geopandas as gpd
import webbrowser
import locale
from branca.element import Figure
import json
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from folium import IFrame
from folium import Map, Popup
from folium.map import Icon, Marker
from folium.features import CustomIcon
from branca.element import Element
import itertools

In [2]:
# Establecer la configuración regional
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Rutas de los archivos
ruta_ayuntamientos = 'C:/Users/franc/OneDrive - INTEC/Escritorio/Proyectos/Python/Gobiernos locales/coordenadas.xlsx'
ruta_gastos = 'C:/Users/franc/OneDrive - INTEC/Escritorio/Proyectos/Python/Gobiernos locales/Gastosos 2.xlsx'
ruta_geojson = 'C:/Users/franc/OneDrive - INTEC/Escritorio/Proyectos/Python/Gobiernos locales/PROVINCIAS.geojson'
ruta_search = 'C:/Users/franc/OneDrive - INTEC/Escritorio/Proyectos/Python/Gobiernos locales/buscador.geojson'

In [3]:
#Lector de los archivos
search_data = gpd.read_file(ruta_search)
# Convert GeoJSON data to JSON
search_json = search_data.to_json()
# ARCHIVO AYUNTAMIENTOS
ayuntamiento_df = pd.read_excel(ruta_ayuntamientos)
# ARCHIVO GASTOS
gastos_df = pd.read_excel(ruta_gastos)

In [4]:
# Preparar los datos de ayuntamientos
ayuntamiento_df['Nombre del ayuntamiento'] = ayuntamiento_df['Nombre del ayuntamiento'].str.strip()

# Preparar los datos de gastos
gastos_df = gastos_df.rename(columns={'GASTOS': 'MONTO'})
gastos_df['AYUNTAMIENTO'] = gastos_df['AYUNTAMIENTO'].str.strip()
gastos_df['PROVINCIA'] = gastos_df['PROVINCIA'].str.upper().str.strip()


#> Agrupar por provincia y sumar los gastos
total_gastos_por_provincia = gastos_df.groupby('PROVINCIA')['MONTO'].sum().reset_index()

#> Cargar el archivo GeoJSON
prov = gpd.read_file(ruta_geojson)
prov['TOPONIMIA'] = prov['TOPONIMIA'].str.upper().str.strip()

#> Fusionar los totales de gastos con las provincias
prov = prov.merge(total_gastos_por_provincia, left_on='TOPONIMIA', right_on='PROVINCIA', how='left')
prov['MONTO'] = prov['MONTO'].fillna(0)

# Coordenadas iniciales del mapa
initial_location = [18.5019667, -69.8789901]

# Crear un mapa de Folium
mapa = folium.Map(location=initial_location, zoom_start=9)

# Crear MarkerCluster
marker_cluster = MarkerCluster(name='Ayuntamientos').add_to(mapa)

#Filtro de CONCEPTO
gastos_por_concepto = gastos_df.groupby(['AYUNTAMIENTO', 'CONCEPTO'])['MONTO'].sum().reset_index()




In [5]:
import json

# Marker Cluster

for index, row in ayuntamiento_df.iterrows():
    ayuntamiento_gastos = gastos_por_concepto[gastos_por_concepto['AYUNTAMIENTO'] == row['Nombre del ayuntamiento']]
    gastos_html = "<ul>"
    for _, gasto_row in ayuntamiento_gastos.iterrows():
        gastos_html += f"<li>{gasto_row['CONCEPTO']}: ${gasto_row['MONTO']:,.2f}</li>"
    gastos_html += "</ul>"

    total_gastos = ayuntamiento_gastos['MONTO'].sum()
    gastos_html_escaped = gastos_html.replace("'", "\\'").replace('"', '\\"')

    # Lista de URLs de imágenes
    image_urls = [
        f"graphs_ayuntamientos/{row['ID']}.jpg",
        f"graphs_ayuntamientos_boxplot2/{row['ID']}.jpg",
        f"graphs_ayuntamientos_boxplotParte3/{row['ID']}.jpg",
        f"graphs_ayuntamientos_boxplotParte4/{row['ID']}.jpg",
        f"graphs_ayuntamientos_CONCEPTO/{row['ID']}.jpg",
        f"graphs_ayuntamientos_Pareto/{row['ID']}.jpg",
    ]
    image_urls_json = json.dumps(image_urls)  

    # Contenido del popup con el botón para ver más detalles

   
    popup_content = f"""
    <div class='popup-content' style='width: 250px; font-size: 16px;'>
        <b>Ayuntamiento:</b> {row['Nombre del ayuntamiento']}<br>
        <b>Total Gastos:</b> ${total_gastos:,.2f}<br>
        <img src='{image_urls[0]}' width='250' height='250' alt='Imagen del Ayuntamiento' onclick='window.parent.showImageModal({image_urls_json});'>
        <div style='text-align: center; margin-top: 10px;'>  <!-- Añadir margen superior y centrar el texto -->
            <a href='#' style='font-size: 18px;' data-gastos='{gastos_html_escaped}' onclick='window.parent.updateInfo(this)'>Ver gastos por concepto</a>
        </div>
    </div>
    """


    # Crear el marcador y agregarlo al mapa
    marker = folium.Marker(
        location=[row['latitud'], row['longitud']],
        popup=folium.Popup(popup_content, max_width=500),
        icon=folium.Icon(color="beige", icon="landmark", prefix='fa')
    )
    marker.add_to(marker_cluster)


In [6]:

bright_colors = [
    "#FF0000",  # Rojo
    "#00FF00",  # Verde
    "#0000FF",  # Azul
    "#FFFF00",  # Amarillo
    "#FF00FF",  # Magenta
    "#00FFFF",  # Cian
    "#FF8000",  # Naranja
    "#FF0080",  # Rosa fuerte
    "#800080",  # Púrpura
    "#800000",  # Marrón
    "#008080",  # Verde azulado
    "#808000",  # Oliva
    "#FFA500",  # Naranja (color zanahoria)
    "#FF4500",  # Naranja rojizo
    "#DA70D6",  # Orquídea
    "#EEE8AA",  # Pálido dorado
    "#FA8072",  # Salmón
    "#DAA520",  # Oro viejo
    "#40E0D0",  # Turquesa
    "#1E90FF",  # Azul dodger
    "#C71585",  # Rojo violeta medio
    "#D2691E",  # Chocolate
    "#00FA9A",  # Verde medio primavera
    "#48D1CC",  # Turquesa medio
    "#C0C0C0",  # Plata
    "#FFD700",  # Oro
    "#F0E68C",  # Caqui
    "#E6E6FA",  # Lavanda
    "#FFF0F5",  # Lavanda rubor
    "#7CFC00",  # Césped verde
    "#FF69B4",  # Rosa caliente
    "#BA55D3"   # Orquídea media
]


# Asegurarse de que hay suficientes colores para las provincias
assert len(bright_colors) >= len(prov['PROV'].unique())

# Crear un iterador de colores
color_iterator = itertools.cycle(bright_colors)

# Asignar un color de la paleta a cada provincia
colores_provincia = {prov_id: next(color_iterator) for prov_id in prov['PROV'].unique()}




# Función de resaltado personalizada para las provincias
def highlight_feature_provincia(feature):
    return {
        'weight': 3,
        'color': 'black',
        'dashArray': '3',
        'opacity': 0.8,
        'fillColor': '#F7FE2E'
    }


# Función de resaltado personalizada para las provincias
def highlight_feature_provincia(feature):
    return {
        'weight': 3,
        'color': 'black',
        'dashArray': '3',
        'opacity': 0.8,
        'fillColor': '#F7FE2E'
    }

# Función de estilo que usa los colores asignados
def style_function(feature):
    prov_id = feature['properties']['PROV']
    return {
        'fillColor': colores_provincia.get(prov_id, '#FFFFFF'),  
        'color': 'black',
        'weight': 2,
        'opacity': 0.7
    }

# Función para tooltips personalizados
def custom_tooltip(feature):
    provincia = feature['properties']['TOPONIMIA']
    prov_id = feature['properties']['PROV']
    image_url = f"graphs/{str(prov_id).zfill(2)}.png"
    return f"""
        <div>
            <h4>Provincia: {provincia}</h4>
            <img src="{image_url}" width="150">
        </div>
    """
# Convertir 'prov' a GeoJSON
geojson_prov = prov.to_json()


geojson_provincias = folium.GeoJson(
    prov,
    name='Provincias',
    style_function=style_function,
    highlight_function=highlight_feature_provincia,
    smooth_factor=2.0,
    show=True,
    control=True,
    tooltip=folium.GeoJsonTooltip(
        fields=['TOPONIMIA', 'MONTO'],  
        aliases=['Provincia:', 'Total Gastos:'],
        style='background-color: white; color: #333333; font-family: Arial; font-size: 16px; padding: 10px;',
        localize=True
    )
).add_to(mapa)

In [7]:
# Añadir la capa GeoJson para los ayuntamientos pero no la hagas visible por defecto
ayuntamientos_layer = folium.GeoJson(
    search_json,
    name='Marcado',
    show=False  
).add_to(mapa)

# Añadir el control de búsqueda
search = Search(
    layer=ayuntamientos_layer,
    name='Marcador',
    geom_type='Point', 
    placeholder="Search for an Ayuntamiento",
    search_label='Nombre del ayuntamiento',
).add_to(mapa)

In [8]:
# Crea el control de capas pero no lo añades al mapa aún
layer_control = folium.LayerControl()

# Añadir capas y control de capas
folium.TileLayer('OpenStreetMap', name='Mapa Callejero').add_to(mapa)
folium.TileLayer('cartodbdark_matter', name='Mapa Oscuro').add_to(mapa)
folium.TileLayer('esriworldstreetmap', name='Mapa Esri').add_to(mapa)


# Crear el control de capas antes de agregar la capa 'Marcado'
layer_control = folium.LayerControl('topright', collapsed=True)
layer_control.add_to(mapa)



In [9]:
# Ruta donde deseas guardar el archivo
ruta_destino = 'C:\\Users\\franc\\OneDrive - INTEC\\Escritorio\\Proyectos\\Python\\Gobiernos locales\\Mapa_Gasto.html'

# Guardar el mapa en la ruta especificada
mapa.save(ruta_destino)

# Abrir el archivo guardado
webbrowser.open('file://' + os.path.realpath(ruta_destino))


True